In [ ]:
import dask.array as da
from pathlib import Path
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader

In [ ]:
base_directory = Path("/media/starfish/LargeSSD/data/cryoET/data/segmentation")
actin_filename = base_directory / "00004_actin_ground_truth_zarr"
microtubules_filename = base_directory / "00004_MT_ground_truth_zarr"

In [ ]:
def load_data(input_filepath: Path) -> da.Array:
    """Load the OME-Zarr data and return a dask array"""
    url = parse_url(input_filepath)
    reader = Reader(url)
    nodes = list(reader())
    image_node = nodes[0]
    dask_data = image_node.data[0]
    return dask_data.persist()

In [ ]:
dask_data = load_data(actin_filename)

In [ ]:
dask_data

In [ ]:
dask_data.blocks[0, 0, 0]

In [ ]:
chunk_layout = dask_data.chunks

In [ ]:
chunk_layout

In [ ]:
def iterate_chunks(dask_data):
    chunks = []
    dimensions = []
    chunk_layout = dask_data.chunks
    for zi, z in enumerate(chunk_layout[0]):
        for yi, y in enumerate(chunk_layout[1]):
            for xi, x in enumerate(chunk_layout[2]):
                chunk = dask_data.blocks[zi, yi, xi]
                chunks.append(chunk) 
                dimensions.append((
                    (sum(list(chunk_layout[0])[:zi]), sum(list(chunk_layout[1][:yi])), sum(list(chunk_layout[2][:xi]))),
                    (sum(list(chunk_layout[0])[:zi+1]), sum(list(chunk_layout[1][:yi+1])), sum(list(chunk_layout[2][:xi+1])))
                                 ))
    return chunks, dimensions

In [ ]:
chunks, dimensions = iterate_chunks_r(dask_data)

In [ ]:
for chunk, dimension in zip(chunks, dimensions):
    print(chunk)
    print(dimension)

In [ ]:
chunks[-1]

In [ ]:
def iterate_chunks_r(dask_data):
    chunks = []
    dimensions = []
    chunk_layout = dask_data.chunks

    for zi, z in enumerate(chunk_layout[0]):
        for yi, y in enumerate(chunk_layout[1]):
            for xi, x in enumerate(chunk_layout[2]):
                chunk = dask_data.blocks[zi, yi, xi]
                chunks.append(chunk)

                # Calculate the chunk dimensions
                start = (sum(chunk_layout[0][:zi]), sum(chunk_layout[1][:yi]), sum(chunk_layout[2][:xi]))
                end = (start[0] + z, start[1] + y, start[2] + x)
                dimensions.append((start, end))

    return chunks, dimensions